https://www.kaggle.com/ironfrown/deep-learning-house-price-prediction-keras

In [1]:
from __future__ import print_function
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import keras
from keras import metrics
from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import Adam, RMSprop
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from keras.utils import plot_model
from keras.models import load_model

D:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [17]:
#kc_data = pd.read_csv(r'./input/kc_house_data.csv')
kc_data = pd.read_pickle('df18featuesy.p')

In [18]:
# kc_data_org['sale_yr'] = pd.to_numeric(kc_data_org.date.str.slice(0, 4))
# kc_data_org['sale_month'] = pd.to_numeric(kc_data_org.date.str.slice(4, 6))
# kc_data_org['sale_day'] = pd.to_numeric(kc_data_org.date.str.slice(6, 8))

# kc_data = pd.DataFrame(kc_data_org, columns=[
#         'sale_yr','sale_month','sale_day',
#         'bedrooms','bathrooms','sqft_living','sqft_lot','floors',
#         'condition','grade','sqft_above','sqft_basement','yr_built',
#         'zipcode','lat','long','sqft_living15','sqft_lot15','price'])

label_col = 'price'

print(kc_data.describe())

        RDEATH2016  RNATURALINC2016  RINTERNATIONALMIG2016  RDOMESTICMIG2016  \
count  8757.000000      8757.000000            8757.000000       8757.000000   
mean      9.850174         1.614091               1.219975          0.030908   
std       2.426470         3.892970               1.663726          8.960091   
min       1.522920       -12.437011              -0.857082        -68.325997   
25%       8.256732        -0.815895               0.243078         -4.889880   
50%       9.880820         1.351976               0.661376         -0.612531   
75%      11.493437         3.787800               1.548359          4.480376   
max      21.594684        27.051161              17.903389         72.957977   

        RBIRTH2016        HPIUS    mortgrate        unemp          cpi  \
count  8757.000000  8757.000000  8757.000000  8757.000000  8757.000000   
mean     11.464265     0.003773    -0.009783    -0.008234     0.001952   
std       2.347001     0.003826     0.019136     0.016942

In [19]:
kc_data.head()

,RDEATH2016,RNATURALINC2016,RINTERNATIONALMIG2016,RDOMESTICMIG2016,RBIRTH2016,HPIUS,mortgrate,unemp,cpi,AL,...,RNATURALINC2016 (t-2),RINTERNATIONALMIG2016 (t-2),RDOMESTICMIG2016 (t-2),RBIRTH2016 (t-2),price (t-2),HPIUS (t-2),mortgrate (t-2),unemp (t-2),cpi (t-2),price
Year,,,,,,,,,,,,,,,,,,,,,
2011-01-01,11.632586,0.621868,-0.146322,-1.133994,12.254454,-0.002834,0.010616,-0.021505,0.004763,1,...,2.345519,0.327282,5.836523,11.563952,-0.024568,-0.002834,0.010616,-0.021505,0.004763,-0.077495
2011-01-01,9.871136,3.058662,-0.052136,1.807391,12.929798,-0.002834,0.010616,-0.021505,0.004763,1,...,1.962947,0.989585,14.768069,11.831760,-0.093731,-0.002834,0.010616,-0.021505,0.004763,-0.086218
2011-01-01,12.950211,-1.174622,0.910332,-2.496072,11.775588,-0.002834,0.010616,-0.021505,0.004763,1,...,0.621868,-0.146322,-1.133994,12.254454,-0.077495,-0.002834,0.010616,-0.021505,0.004763,-0.194399
2011-01-01,11.911623,-3.419789,0.076849,6.109510,8.491835,-0.002834,0.010616,-0.021505,0.004763,1,...,3.058662,-0.052136,1.807391,12.929798,-0.086218,-0.002834,0.010616,-0.021505,0.004763,-0.148970
2011-01-01,10.092682,3.397340,1.728471,0.993374,13.490022,-0.002834,0.010616,-0.021505,0.004763,1,...,-1.174622,0.910332,-2.496072,11.775588,-0.194399,-0.002834,0.010616,-0.021505,0.004763,-0.076972


In [20]:
def train_validate_test_split(df, train_part=.6, validate_part=.2, test_part=.2, seed=None):
    np.random.seed(seed)
    total_size = train_part + validate_part + test_part
    train_percent = train_part / total_size
    validate_percent = validate_part / total_size
    test_percent = test_part / total_size
    perm = np.random.permutation(df.index)
    m = len(df)
    train_end = int(train_percent * m)
    validate_end = int(validate_percent * m) + train_end
    train = perm[:train_end]
    validate = perm[train_end:validate_end]
    test = perm[validate_end:]
    return train, validate, test

In [21]:
train_size, valid_size, test_size = (70, 30, 0)
kc_train, kc_valid, kc_test = train_validate_test_split(kc_data, 
                              train_part=train_size, 
                              validate_part=valid_size,
                              test_part=test_size,
                              seed=2017)

In [22]:
kc_y_train = kc_data.loc[kc_train, [label_col]]
kc_x_train = kc_data.loc[kc_train, :].drop(label_col, axis=1)
kc_y_valid = kc_data.loc[kc_valid, [label_col]]
kc_x_valid = kc_data.loc[kc_valid, :].drop(label_col, axis=1)

print('Size of training set: ', len(kc_x_train))
print('Size of validation set: ', len(kc_x_valid))
print('Size of test set: ', len(kc_test), '(not converted)')

Size of training set:  9017747
Size of validation set:  3857476
Size of test set:  1 (not converted)


In [23]:
def norm_stats(df1, df2):
    dfs = df1.append(df2)
    minimum = np.min(dfs)
    maximum = np.max(dfs)
    mu = np.mean(dfs)
    sigma = np.std(dfs)
    return (minimum, maximum, mu, sigma)

In [24]:
def z_score(col, stats):
    m, M, mu, s = stats
    df = pd.DataFrame()
    for c in col.columns:
        df[c] = (col[c]-mu[c])/s[c]
    return df

In [25]:
stats = norm_stats(kc_x_train, kc_x_valid)
arr_x_train = np.array(z_score(kc_x_train, stats))
arr_y_train = np.array(kc_y_train)
arr_x_valid = np.array(z_score(kc_x_valid, stats))
arr_y_valid = np.array(kc_y_valid)

print('Training shape:', arr_x_train.shape)
print('Training samples: ', arr_x_train.shape[0])
print('Validation samples: ', arr_x_valid.shape[0])

Training shape: (9017747, 79)
Training samples:  9017747
Validation samples:  3857476


In [26]:
pd.DataFrame(arr_x_train).describe()

,0,1,2,3,4,5,6,7,8,9,...,69,70,71,72,73,74,75,76,77,78
count,9.017747e+06,9.017747e+06,9.017747e+06,9.017747e+06,9.017747e+06,9.017747e+06,9.017747e+06,9.017747e+06,9.017747e+06,9.017747e+06,...,9.017747e+06,9.017747e+06,9.017747e+06,9.017747e+06,9.017747e+06,9.017747e+06,9.017747e+06,9.017747e+06,9.017747e+06,9.017747e+06
mean,3.659216e-04,-2.797203e-04,-4.130255e-05,6.094169e-04,-8.558740e-05,-9.801214e-04,-2.493424e-03,-8.677850e-03,3.432867e-03,4.820808e-05,...,5.991297e-04,-2.791303e-04,-4.635210e-05,6.111175e-04,-7.922623e-05,1.020143e-03,-9.722583e-04,-2.498436e-03,-8.652522e-03,3.413229e-03
std,1.000045e+00,1.000036e+00,9.997072e-01,1.000901e+00,9.999152e-01,9.972574e-01,9.958553e-01,9.966305e-01,9.952755e-01,1.000154e+00,...,1.000860e+00,1.000037e+00,9.997098e-01,1.000898e+00,9.999189e-01,9.997258e-01,9.972605e-01,9.958691e-01,9.966412e-01,9.952903e-01
min,-3.436756e+00,-3.607960e+00,-1.249595e+00,-7.579588e+00,-3.233829e+00,-1.800122e+00,-1.448069e+00,-9.020263e-01,-1.907935e+00,-1.524504e-01,...,-6.994316e+00,-3.608432e+00,-1.249915e+00,-7.579834e+00,-3.234048e+00,-5.339462e+00,-1.799079e+00,-1.448310e+00,-9.020570e-01,-1.907941e+00
25%,-6.554872e-01,-6.247605e-01,-5.883721e-01,-5.502265e-01,-6.295247e-01,-9.814363e-01,-6.019886e-01,-7.830451e-01,-3.341023e-02,-1.524504e-01,...,-5.623686e-01,-6.247533e-01,-5.884198e-01,-5.505319e-01,-6.294399e-01,-6.011989e-01,-9.806910e-01,-6.023415e-01,-7.830791e-01,-3.360180e-02
50%,1.390582e-02,-6.805552e-02,-3.351558e-01,-7.400682e-02,-5.440144e-02,2.662997e-01,6.019152e-02,-6.945529e-01,3.510465e-01,-1.524504e-01,...,-7.954433e-02,-6.770179e-02,-3.352111e-01,-7.440974e-02,-5.424955e-02,1.295969e-01,2.665911e-01,5.975077e-02,-6.945892e-01,3.508170e-01
75%,6.786480e-01,5.582606e-01,1.977197e-01,4.971000e-01,5.212230e-01,6.046289e-01,1.146673e+00,1.225233e+00,7.763853e-01,-1.524504e-01,...,4.821305e-01,5.584578e-01,1.975679e-01,4.969797e-01,5.213969e-01,7.278314e-01,6.047973e-01,1.146088e+00,1.225145e+00,7.761139e-01
max,4.836428e+00,6.540324e+00,1.000566e+01,8.076581e+00,7.926121e+00,1.460127e+00,1.529243e+00,1.530847e+00,8.834030e-01,6.559511e+00,...,7.934375e+00,6.541482e+00,1.000664e+01,8.075723e+00,7.927203e+00,1.758029e+00,1.459984e+00,1.528607e+00,1.530750e+00,8.831210e-01


In [27]:
def basic_model_1(x_size, y_size):
    t_model = Sequential()
    t_model.add(Dense(100, activation="tanh", input_shape=(x_size,)))
    t_model.add(Dense(50, activation="relu"))
    t_model.add(Dense(y_size))
    print(t_model.summary())
    t_model.compile(loss='mean_squared_error',
        optimizer=Adam(),
        metrics=[metrics.mae])
    return(t_model)

In [16]:
def basic_model_2(x_size, y_size):
    t_model = Sequential()
    t_model.add(Dense(100, activation="tanh", input_shape=(x_size,)))
    t_model.add(Dropout(0.1))
    t_model.add(Dense(50, activation="relu"))
    t_model.add(Dense(20, activation="relu"))
    t_model.add(Dense(y_size))
    print(t_model.summary())
    t_model.compile(loss='mean_squared_error',
        optimizer=Adam(),
        metrics=[metrics.mae])
    return(t_model)

In [ ]:
def basic_model_3(x_size, y_size):
    t_model = Sequential()
    t_model.add(Dense(80, activation="tanh", kernel_initializer='normal', input_shape=(x_size,)))
    t_model.add(Dropout(0.2))
    t_model.add(Dense(120, activation="relu", kernel_initializer='normal', 
        kernel_regularizer=regularizers.l1(0.01), bias_regularizer=regularizers.l1(0.01)))
    t_model.add(Dropout(0.1))
    t_model.add(Dense(20, activation="relu", kernel_initializer='normal', 
        kernel_regularizer=regularizers.l1_l2(0.01), bias_regularizer=regularizers.l1_l2(0.01)))
    t_model.add(Dropout(0.1))
    t_model.add(Dense(10, activation="relu", kernel_initializer='normal'))
    t_model.add(Dropout(0.0))
    t_model.add(Dense(y_size))
    t_model.compile(
        loss='mean_squared_error',
        optimizer='nadam',
        metrics=[metrics.mae])
    return(t_model)

In [28]:
model = basic_model_1(arr_x_train.shape[1], arr_y_train.shape[1])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 100)               8000      
_________________________________________________________________
dense_2 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 51        
Total params: 13,101
Trainable params: 13,101
Non-trainable params: 0
_________________________________________________________________
None
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 100)               8000      
_________________________________________________________________
dense_2 (Dense)              (None, 50)                5050      
_________________________________________________________________
d

In [29]:
epochs = 5
batch_size = 5

print('Epochs: ', epochs)
print('Batch size: ', batch_size)

Epochs:  5
Batch size:  5


In [30]:
keras_callbacks = [
    # ModelCheckpoint('/tmp/keras_checkpoints/model.{epoch:02d}-{val_loss:.2f}.hdf5', monitor='val_loss', save_best_only=True, verbose=2)
    # ModelCheckpoint('/tmp/keras_checkpoints/model.{epoch:02d}.hdf5', monitor='val_loss', save_best_only=True, verbose=0)
    # TensorBoard(log_dir='/tmp/keras_logs/model_3', histogram_freq=0, write_graph=True, write_images=True, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None),
    EarlyStopping(monitor='val_mean_absolute_error', patience=20, verbose=0)
]

In [31]:
model.fit(arr_x_train, arr_y_train,
    batch_size=batch_size,
    epochs=epochs,
    shuffle=True,
    verbose=0, # Change it to 2, if wished to observe execution
    validation_data=(arr_x_valid, arr_y_valid),
    callbacks=keras_callbacks)

KeyboardInterrupt: 

In [ ]:
train_score = model.evaluate(arr_x_train, arr_y_train, verbose=0)
valid_score = model.evaluate(arr_x_valid, arr_y_valid, verbose=0)

print('Train MAE: ', round(train_score[1], 4), ', Train Loss: ', round(train_score[0], 4)) 
print('Val MAE: ', round(valid_score[1], 4), ', Val Loss: ', round(valid_score[0], 4))

In [ ]:
def plot_hist(h, xsize=6, ysize=10):
    # Prepare plotting
    fig_size = plt.rcParams["figure.figsize"]
    plt.rcParams["figure.figsize"] = [xsize, ysize]
    fig, axes = plt.subplots(nrows=4, ncols=4, sharex=True)
    
    # summarize history for MAE
    plt.subplot(211)
    plt.plot(h['mean_absolute_error'])
    plt.plot(h['val_mean_absolute_error'])
    plt.title('Training vs Validation MAE')
    plt.ylabel('MAE')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    
    # summarize history for loss
    plt.subplot(212)
    plt.plot(h['loss'])
    plt.plot(h['val_loss'])
    plt.title('Training vs Validation Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    
    # Plot it all in IPython (non-interactive)
    plt.draw()
    plt.show()

    return


In [ ]:
plot_hist(history.history, xsize=8, ysize=12)


In [ ]:
y_pred= model.predict(kc_x_valid)
y_pred

In [ ]:
y_true = arr_y_valid

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
r2_score(y_true, y_pred)

In [ ]:
r2_score(y_true, y_pred, multioutput='variance_weighted')

In [ ]:
#from sklearn.metrics import classification_report,confusion_matrix


In [ ]:
from sklearn.metrics import explained_variance_score
explained_variance_score(y_true, y_pred) 